Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [328]:
import pandas as pd
import numpy as np
import warnings
from pandas.core.series import Series
from sklearn.preprocessing import FunctionTransformer

def replacement(columnSeris:Series, l:list):
    """
    Функция производит замену в структуре Series, параметр columnSeris, что искать и что менять определяется параметром l, который содержит кортежи со значениями для поиска и замены.
    Первый элемент кортежа, это то, что подлежит замене, второй элемент — это то, чем будет заменена.    
    :return:Series
    """
    columnSerisRes = columnSeris
    for item in l:        
        columnSerisRes[columnSerisRes == item[0]] = item[1]
    return columnSerisRes


warnings.filterwarnings('ignore')
df = pd.read_csv("./web1_/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [194]:
gender = df["gender"].value_counts()
amountMale = gender['Male']
amountFemale = gender['Female']
ratio = amountMale/amountFemale
print(f'Мужчин: {amountMale}, женщин: {amountFemale}, соотношение мужчин и женщин: {ratio:.3f}')

Мужчин: 3555, женщин: 3488, соотношение мужчин и женщин: 1.019


##### 2. Какое количество уникальных значений у поля InternetService?

In [195]:
df["InternetService"].nunique()
print(f'Количество уникальных значений у поля InternetService: {df["InternetService"].nunique()}')

Количество уникальных значений у поля InternetService: 3


In [196]:
df["InternetService"].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [198]:
totalCharges = df["TotalCharges"]
totalCharges[totalCharges == " "] = np.nan
describe = totalCharges.describe()
describe["median"] = totalCharges.median()
describe

count         7032
unique        6530
top           20.2
freq            11
median    1397.475
Name: TotalCharges, dtype: object

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [199]:
df["PhoneService"] = pd.to_numeric(replacement(df["PhoneService"], l=[('Yes',1),('No',0)]))

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [200]:
totalCharges[totalCharges == " "] = np.nan
totalCharges = totalCharges.astype(np.float32)
totalCharges = totalCharges.fillna(value=0)
df["TotalCharges"] = totalCharges
describe = df["TotalCharges"].describe()
describe["median"] = totalCharges.median()
describe

count     7043.000000
mean      2279.732178
std       2266.794434
min          0.000000
25%        398.549988
50%       1394.550049
75%       3786.599976
max       8684.799805
median    1394.550049
Name: TotalCharges, dtype: float64

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [201]:
df["Churn"] = pd.to_numeric(replacement(df["Churn"], l=[('Yes',1),('No',0)]))

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [202]:
listColumn = ['StreamingMovies', 'StreamingTV', 'TechSupport' ]
for item in listColumn:
    df[item] = pd.to_numeric(replacement(df[item], l=[('Yes',1),('No',0),('No internet service',0)]))

In [203]:
df["TechSupport"].unique()

array([0, 1], dtype=int64)

In [204]:
df["StreamingMovies"].unique()

array([0, 1], dtype=int64)

In [205]:
df["TechSupport"].unique()

array([0, 1], dtype=int64)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [206]:
df["PhoneService"].ffill(0)

0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: int64

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [233]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

columns = set(columns)
allColumns = set(df.columns)
df.drop(allColumns.difference(columns), axis=1, inplace=True)
df

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
0,Female,1,No,No,No,No,29.85,No
1,Male,34,Yes,No,No,No,1889.5,No
2,Male,2,Yes,No,No,No,108.15,Yes
3,Male,45,No,Yes,No,No,1840.75,No
4,Female,2,Yes,No,No,No,151.65,Yes
...,...,...,...,...,...,...,...,...
7038,Male,24,Yes,Yes,Yes,Yes,1990.5,No
7039,Female,72,Yes,No,Yes,Yes,7362.9,No
7040,Female,11,No,No,No,No,346.45,No
7041,Male,4,Yes,No,No,No,306.6,Yes


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [215]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X = df
y = df['Churn']
X.drop('Churn', axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [371]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):        
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        return self

    def transform(self, X):    
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]        
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [392]:
gender.fit_transform(X)

,gender_Female,gender_Male
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
7038,0,1
7039,1,0
7040,1,0
7041,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [377]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [384]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [386]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [387]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])

feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [400]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [407]:
y_pred_proba = pipeline.predict_proba(X_test)
y_pred_proba

array([[0.29, 0.71],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.93, 0.07],
       [0.23, 0.77],
       [0.94, 0.06]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [412]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score(y_test, y_pred_proba[:, 1])

0.7920710765169473

In [413]:
log_loss(y_test, y_pred_proba[:, 1])

0.7093278035153542

### Сохраним наш пайплайн

In [ ]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)